In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import xml.etree.ElementTree as ET
from pathlib import Path
import json


In [2]:
def get_soup_for_ecmdb_ID(ecmdb_id):
    response = requests.get(f'https://ecmdb.ca/compounds/{ecmdb_id}')
    soup = BeautifulSoup(response.text, 'lxml-xml')
    xml_string = soup.find('concentrations')
    return xml_string

In [3]:
def parse_concentrations_ecmdb(soup):
    # Convert the soup to a string for parsing
    tree = ET.fromstring(str(soup))
    
    # Initialize a list to store extracted data
    data = []
    
    # Iterate over all elements in the XML and collect their tags and text values
    index = 0
    rows = []
    row = {}
    for element in tree.iter():
        text = element.text.strip() if element.text else ''
        if not len(text):
            continue
        if row.get(element.tag):
            print("New!")
            rows.append(row)
            row = {}
        row[element.tag] = text
    rows.append(row)
    # Convert the data into a DataFrame for tabular display
    df = pd.DataFrame(rows)
    
    return df

In [4]:
def get_ecmdb_conc(ecmdb_id):
    xml_string = get_soup_for_ecmdb_ID(ecmdb_id)
    return parse_concentrations_ecmdb(xml_string)

# Get ECMDB id for all metabolites

In [5]:

data_folder = Path('../../data')
met_info_fn = data_folder / 'this_project' / '5_div' / '5C_metabolite_info.csv'
met_info = pd.read_csv(met_info_fn, index_col=0)

In [6]:
ecmdb_fn = data_folder / 'Other' / 'ecmdb.json' 
with open(ecmdb_fn, 'r') as f:
    ecmdb = json.load(f)

In [7]:
ecm_df = pd.DataFrame(ecmdb)

# Used to get ECMDB IDs from names matched by Gemini

In [20]:
ecmdb_mapping_fn = 'metabolite_to_ecmdb.csv'
conc_df = pd.read_csv(ecmdb_mapping_fn)

In [21]:
ecmdb_id = []
for i, row in conc_df.iterrows():
    name = row['ECMDB name']
    dfi = ecm_df.loc[ecm_df.name==name]
    if len(dfi)==1:
        ecmdb_id.append(dfi['met_id'].values[0])
    else:
        ecmdb_id.append(None)
conc_df['ECMDB ID'] = ecmdb_id

In [50]:
conc_df.to_csv(ecmdb_mapping_fn)

# Extract all concentration data from ECMDB for relecant metabolites

In [33]:
conc_data = {}
for i, row in conc_df.iterrows():
    if row['ECMDB ID']:
        conc_dfi = get_ecmdb_conc(row['ECMDB ID'])
        if conc_dfi.size:
            conc_data[row['Metabolite']] = conc_dfi

New!
New!
New!
New!
New!
New!
New!
New!
New!
New!
New!
New!
New!
New!
New!
New!
New!
New!
New!
New!
New!
New!
New!
New!
New!
New!
New!
New!
New!
New!
New!
New!
New!
New!
New!
New!
New!
New!
New!
New!
New!
New!
New!
New!
New!
New!
New!
New!
New!
New!
New!
New!
New!
New!
New!
New!
New!
New!
New!
New!
New!
New!
New!
New!
New!
New!
New!
New!
New!
New!
New!
New!
New!
New!
New!
New!
New!
New!
New!
New!
New!
New!
New!
New!
New!
New!
New!
New!
New!
New!
New!
New!
New!
New!
New!
New!
New!
New!
New!
New!
New!
New!
New!
New!
New!
New!
New!
New!
New!
New!
New!
New!
New!
New!
New!
New!
New!
New!
New!
New!
New!
New!
New!


In [36]:
all_conc = pd.concat(conc_data).reset_index()

In [38]:
all_conc.rename(columns={'level_0': 'Metabolite', 'level_1': '#'}, inplace=True)

In [49]:
all_conc_fn = data_folder / 'this_project' / '5_div' / '5F_intracellular_concentrations.csv'
all_conc.to_csv(all_conc_fn)